In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from dateutil import parser as dparser
import urllib.parse
import timeit
import os
import numpy as np
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy import sparse
from datasets import load_dataset, DatasetDict, load_metric

import torch
from transformers import AutoModelForSequenceClassification, DistilBertTokenizer
from transformers import pipeline

from util.config import config
from util.pyRanker import BM25
from util.web_query_v2 import web_query
from util.ticker import Ticker
from util.data_manager import data_manager
from util.corpus import Corpus
import ipywidgets as widgets

# Re-Running Sentiment through a different model
Below is an example of using the existing collected data in the _data directory to establish a list of urls and re-collect and re-run the sentiment analysis.

This is useful if you would like to use the framework with a different sentiment model

In the training folder is an example of how we fine tuned the model that we use by default in the corpus.

In [ ]:
#defining a new model to be used
model = AutoModelForSequenceClassification.from_pretrained("adp12/cs410finetune1")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
classifier = pipeline(task='sentiment-analysis',model=model,tokenizer=tokenizer)
max_tokens = int(tokenizer.model_max_length)

In [3]:
cfig = config()

In [4]:
datadir = os.path.join(os.getcwd(),"_data")
manager = data_manager(datadir=datadir)

In [12]:
#dictionary of ticker symbols and prime_query selections

qs = {'AMC':'ticker','BIDU':'name',
      'CAT':'name','CCL':'name','GE':'ticker','GM':'both',
      'IBM':'ticker','JNJ':'name','M':'name','MCD':'name',
      'MSFT':'name','NIO':'name','NVDA':'name','PFE':'name',
      'PLUG':'name','QCOM':'name','SNAP':'name','TSLA':'name',
      'VMW':'name','XOM':'name'}

In [13]:
#Collecting a dict of dataframes using the data_manager
dfs = {}
for t in qs.keys():
    df = manager.get_fulldf(ticker=t)
    dfs[t]=df

In [14]:
#looping through ticker dictionary and rebuilding data with new model
for t in qs.keys():
    print(t)
    corpus=Corpus()
    corpus.model=model
    corpus.classifier=classifier
    print('New Corpus Initialized')
    
    tick = Ticker(config=cfig, t=t)
    print('Ticker Created')
    prime_query=qs[t]
    
    wq=web_query(config=cfig)
    wq.results = dfs[t]
    print('Scraping Urls:',len(dfs[t]))
    wq.scrape_results(max_docs='max')

    results = wq.get_results()
    corpus.set_results(results)
    #assign corpus documents as the web query documents and urls
    corpus.set_corpus(documents = wq.documents, urls = wq.urls)

    #Initiating Ranker
    bm25 = BM25(norm='l2', smooth_idf=True, stopwords=corpus.stopwords, sublinear_tf=True)
    bm25.fit(corpus.documents)

    #Building Queries
    corpus.build_queries(ticker=tick, prime=prime_query)
    print('Prime_Query:',corpus.prime_q)
    #Ranking and Pruning
    corpus.rank_docs(ranker=bm25)
    corpus.prune_docs()

    #Sub-Dividing
    print('Sub-Dividing Documents')
    #corpus.sub_divide(tokenizer=tokenizer, cutoff=2, method='sen')
    corpus.sub_divide(cutoff=2, method='sen')
    print('Sub-docs:',len(corpus.sub_list))
    print('Source count:', len(corpus.sub_docs))

    #Ranking and Pruning
    corpus.rank_subdocs(ranker=bm25)
    corpus.prune_subdocs()

    print("Number of sub_docs:",len(corpus.sub_list))
    t = np.array(corpus.sub_list_scores)
    z=np.where(t==0.0)
    print("Number zero ranked:", len(z[0]))
    print("percent of useless subdocs:", round((len(z[0])/len(corpus.sub_list))*100,2),"%")

    #Relevant Set
    corpus.make_relevant()
    corpus.rank_relevant(ranker=bm25)
    #shouldn't be any zeros because of the pruned subdocs, but cant hurt
    corpus.prune_relevant(method='finite',cutoff=0.0)
    print('relevant sources:',len(corpus.relevant_set))
    print('relevant sub_docs:',len(corpus.rel_list))

    #Sentiments
    print('Getting Sentiments')
    corpus.get_sentiments()

    #Building Dataset
    print('Build Data')
    corpus.data_preprocess()
    corpus.build_fulldf()
    corpus.build_pricedf(ticker=tick)


    manager.store_data(ticker=tick.ticker, full_df=corpus.full_df, price_df=corpus.price_df)
    manager.get_ticker_list()
    manager.get_data_info(ticker=tick.ticker)

AMC
New Corpus Initialized
Ticker Created
Scraping Urls: 652


IntProgress(value=0, max=652)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Prime_Query: AMC
Sub-Dividing Documents


IntProgress(value=0, max=643)

...

Token indices sequence length is longer than the specified maximum sequence length for this model (1867 > 512). Running this sequence through the model will result in indexing errors


Sub-docs: 3133
Source count: 248
Number of sub_docs: 3133
Number zero ranked: 2678
percent of useless subdocs: 85.48 %
relevant sources: 248
relevant sub_docs: 455
Getting Sentiments


IntProgress(value=0, max=455)

Build Data


C:\Users\Tony\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
